# Базовые трансформации в Spark

### Из чего состоят dataframe

* датафрейм состоит из строк (объект Row)
* строка состоит из колонок (объект Column)
* Row не имеет методов, Column имеет множество методов
* значение в колонке всегда имеет тип и определяется схемой (Schema)
* spark работает с собственным набором типов данных (см. документацию)

### Выражения и колонки

* выражение (`expression`) - это функция, которая преобразует множество колонок в значение для каждой строки dataframe
* в простейшем случае - само значение колонки датафрейма (`col("a")`)
* функция expr() - создает выражение из строки (`expr("a - 5")`)

### Операции с dataframe

![](df_transforms.PNG)

* добавляем или удаляем колонки
* добавляем или удаляем строки
* трансформируем колонку в строку или наоборот
* меняем порядок строк
* dataframe изменить нельзя, можно лишь создать новый

### Информация о dataframe

* schema - атрибут dataframe, содержащий StructType со схемой
* printSchema() - печать схемы в виде дерева
* columns - атрибут dataframe, содержащий список колонок
* describe() - вычисляет count, mean, standard deviation, min и max для всех цифровых колонок dataframe

### Удаление колонок

* select() - выбор (набора) колонок dataframe
* selectExpr() - аналогично, но с использованием выражений
* drop() - удаление колонок dataframe

### Добавление колонок

* withColumn() - добавление колонки
* withColumnRenamed() - переименование колонки
* функция lit()  - литерал (используется для добавления колонок с константными значениями)

### Удаление строк

* filter() - отбор строк dataframe
* where() - синоним 

### Остальные виды операций

* добавление строк - это join() и union()
* сортировку мы рассмотрим в отдельном шаге вместе с агрегатными функциями 

### Практика

Давайте попробуем основные трансформации на наших данных (данные стран мира).

Тренироваться будем в локальном режиме (на рабочем месте) - см. создание spark session.

In [3]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
os.environ["SPARK_HOME"] = "/home/mk/mk_win/projects/SparkEdu/lib/python3.5/site-packages/pyspark"
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "pyspark-shell"

In [4]:
spark = SparkSession.builder.master("local").appName("transf_test").getOrCreate()

Загрузим наши страны

In [5]:
cdf = spark.read.format("csv") \
    .option("mode", "FAILFAST") \
    .option("inferSchema", "true") \
    .option("header","true") \
    .option("path", "data/countries of the world.csv") \
    .load()

**Схема**

ее лучше всего смотреть в виде дерева

In [6]:
cdf.printSchema()

root
 |-- Country: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Area (sq. mi.): integer (nullable = true)
 |-- Pop. Density (per sq. mi.): string (nullable = true)
 |-- Coastline (coast/area ratio): string (nullable = true)
 |-- Net migration: string (nullable = true)
 |-- Infant mortality (per 1000 births): string (nullable = true)
 |-- GDP ($ per capita): integer (nullable = true)
 |-- Literacy (%): string (nullable = true)
 |-- Phones (per 1000): string (nullable = true)
 |-- Arable (%): string (nullable = true)
 |-- Crops (%): string (nullable = true)
 |-- Other (%): string (nullable = true)
 |-- Climate: string (nullable = true)
 |-- Birthrate: string (nullable = true)
 |-- Deathrate: string (nullable = true)
 |-- Agriculture: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Service: string (nullable = true)



**Колонки**

Список колонок - атрибут, отсюда удобно копировать (колонки названы длинно)

In [7]:
cdf.columns

['Country',
 'Region',
 'Population',
 'Area (sq. mi.)',
 'Pop. Density (per sq. mi.)',
 'Coastline (coast/area ratio)',
 'Net migration',
 'Infant mortality (per 1000 births)',
 'GDP ($ per capita)',
 'Literacy (%)',
 'Phones (per 1000)',
 'Arable (%)',
 'Crops (%)',
 'Other (%)',
 'Climate',
 'Birthrate',
 'Deathrate',
 'Agriculture',
 'Industry',
 'Service']

**describe()**

немного неожиданный результат - посчиталось для всех колонок... Не только числовых.

In [8]:
cdf.describe().show()

+-------+------------+--------------------+--------------------+------------------+--------------------------+----------------------------+--------------------+----------------------------------+------------------+------------+-----------------+------------------+------------------+------------------+------------------+------------------+---------+-----------+--------+-------+
|summary|     Country|              Region|          Population|    Area (sq. mi.)|Pop. Density (per sq. mi.)|Coastline (coast/area ratio)|       Net migration|Infant mortality (per 1000 births)|GDP ($ per capita)|Literacy (%)|Phones (per 1000)|        Arable (%)|         Crops (%)|         Other (%)|           Climate|         Birthrate|Deathrate|Agriculture|Industry|Service|
+-------+------------+--------------------+--------------------+------------------+--------------------------+----------------------------+--------------------+----------------------------------+------------------+------------+-------------

**Отбор колонок**

С помощью `select`-а оставим только нужные нам (обратите внимание на результат - схема результирующего датафрейма)

In [9]:
cdf.select('Country','Region','Population')

DataFrame[Country: string, Region: string, Population: int]

**selectExpr()**

могучая функция: мы знаем, что в модуле `pyspark.sql.functions` есть функция конкатенации, смело вызываем ее для создания такого названия страны. Заодно воспользуемся возможностью и переименуем колонку (`as`).

Если мы хотим, чтобы `show` не обрезал вывод - добавляем параметр `False`.

In [10]:
cdf.selectExpr('concat(Country,"in ",Region) as CountryIn').show(3, False)

+----------------------------------------------+
|CountryIn                                     |
+----------------------------------------------+
|Afghanistan in ASIA (EX. NEAR EAST)           |
|Albania in EASTERN EUROPE                     |
|Algeria in NORTHERN AFRICA                    |
+----------------------------------------------+
only showing top 3 rows



**Удаление колонок**

Удалим так понравившиеся нам колонки и посмотрим - что осталось...

In [11]:
cdf.drop('Country','Region','Population').columns

['Area (sq. mi.)',
 'Pop. Density (per sq. mi.)',
 'Coastline (coast/area ratio)',
 'Net migration',
 'Infant mortality (per 1000 births)',
 'GDP ($ per capita)',
 'Literacy (%)',
 'Phones (per 1000)',
 'Arable (%)',
 'Crops (%)',
 'Other (%)',
 'Climate',
 'Birthrate',
 'Deathrate',
 'Agriculture',
 'Industry',
 'Service']

**Добавим литеральную колонку**

В новом датафрейме будет колонка `ONE` со значением 1.

In [12]:
cdf.select('Country','Region','Population').withColumn("ONE",f.lit(1)).columns

['Country', 'Region', 'Population', 'ONE']

**Переименуем колонку**

Уж очень длинно звучит название колонки - давайте укоротим...

In [13]:
cdf.withColumnRenamed('Area (sq. mi.)','AREA').columns

['Country',
 'Region',
 'Population',
 'AREA',
 'Pop. Density (per sq. mi.)',
 'Coastline (coast/area ratio)',
 'Net migration',
 'Infant mortality (per 1000 births)',
 'GDP ($ per capita)',
 'Literacy (%)',
 'Phones (per 1000)',
 'Arable (%)',
 'Crops (%)',
 'Other (%)',
 'Climate',
 'Birthrate',
 'Deathrate',
 'Agriculture',
 'Industry',
 'Service']

**Фильтрация**

Посмотрим, к какому региону приписали в этой табличке нас (пришлось немного поупражняться, чтобы 

In [14]:
cdf.select('Country','Region').filter(f.col('Country').startswith('Russia')).show()

+-------+--------------------+
|Country|              Region|
+-------+--------------------+
|Russia |C.W. OF IND. STATES |
+-------+--------------------+



### Еще немного практики

**range()**

Простейший способ создаиние датафрейма - создает датафрейм с одной числовой колонкой `id` и заполяет его строками указанного в параметре диапазона. Дата инженерам не нужно создавать датафреймы - они их обычно загружают. Но если нужно - вот способ.

In [15]:
df = spark.range(10).show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
+---+



**distinct()**

С помощью `distinct()` посмотрим - какие вообще регионы имеются в нашем датафрейме. Выглядит как-то подозрительно - что там, пробелы в конце что-ли?

In [16]:
cdf.select('Region').distinct().show(truncate=False)

+-----------------------------------+
|Region                             |
+-----------------------------------+
|BALTICS                            |
|C.W. OF IND. STATES                |
|ASIA (EX. NEAR EAST)               |
|WESTERN EUROPE                     |
|NORTHERN AMERICA                   |
|NEAR EAST                          |
|EASTERN EUROPE                     |
|OCEANIA                            |
|SUB-SAHARAN AFRICA                 |
|NORTHERN AFRICA                    |
|LATIN AMER. & CARIB                |
+-----------------------------------+



**length()**

Добавим к имени региона колонку, которая покажет длину строки... Действительно, пробелы

In [17]:
cdf.select('Region').distinct().withColumn('len',f.length('Region')).show(100,truncate=False)

+-----------------------------------+---+
|Region                             |len|
+-----------------------------------+---+
|BALTICS                            |35 |
|C.W. OF IND. STATES                |20 |
|ASIA (EX. NEAR EAST)               |29 |
|WESTERN EUROPE                     |35 |
|NORTHERN AMERICA                   |35 |
|NEAR EAST                          |35 |
|EASTERN EUROPE                     |35 |
|OCEANIA                            |35 |
|SUB-SAHARAN AFRICA                 |35 |
|NORTHERN AFRICA                    |35 |
|LATIN AMER. & CARIB                |23 |
+-----------------------------------+---+



**monotonically_increasing_id(), alias(), replace()**

Ну и напоследок "завернем" такое преобразование:

* добавим `monotonically_increasing_id()` - обратите внимание: это на простом примере так красиво получилось, в жизни туда еще добавится номер партиции, будет не очень красиво, но монотонно возрастающе...
* переименуем колонку с помощью `alias()`
* через `expr()` "укоротим" название региона и переименуем его сразу в код
* словарной заменой заменим часть регионов на их код

In [18]:
regRepl = { 'OCEANIA': 'OCN', 'ASIA (EX. NEAR EAST)': 'ASA' }
cdf.select(f.monotonically_increasing_id().alias('ID'),'Country',f.expr('rtrim(Region) as RegCode')) \
    .replace(regRepl,None,'RegCode') \
    .show()

+---+------------------+-------------------+
| ID|           Country|            RegCode|
+---+------------------+-------------------+
|  0|      Afghanistan |                ASA|
|  1|          Albania |     EASTERN EUROPE|
|  2|          Algeria |    NORTHERN AFRICA|
|  3|   American Samoa |                OCN|
|  4|          Andorra |     WESTERN EUROPE|
|  5|           Angola | SUB-SAHARAN AFRICA|
|  6|         Anguilla |LATIN AMER. & CARIB|
|  7|Antigua & Barbuda |LATIN AMER. & CARIB|
|  8|        Argentina |LATIN AMER. & CARIB|
|  9|          Armenia |C.W. OF IND. STATES|
| 10|            Aruba |LATIN AMER. & CARIB|
| 11|        Australia |                OCN|
| 12|          Austria |     WESTERN EUROPE|
| 13|       Azerbaijan |C.W. OF IND. STATES|
| 14|     Bahamas, The |LATIN AMER. & CARIB|
| 15|          Bahrain |          NEAR EAST|
| 16|       Bangladesh |                ASA|
| 17|         Barbados |LATIN AMER. & CARIB|
| 18|          Belarus |C.W. OF IND. STATES|
| 19|     

**Итого**

Возможности базовых трансформации поистинне неисчерпаемы. 

Изучайте документацию, пробуйте - в этом ноутбуке, создавайте свои.

В этом модуле вам нужно будет преобразовать "наши" данные - в материалах выше содержатся необходимые методы. Джойны и объединения мы рассмотрим в следующем шаге.

In [19]:
spark.stop()